Test notebook for pulling API data

Testing using EtherScan - getting historical gas prices: https://docs.etherscan.io/api-endpoints/gas-tracker#get-daily-average-gas-limit

In [125]:
# Import Packages
import requests
import json
import pandas as pd


Envinroment Loading

In [126]:
from dotenv import load_dotenv
import os

def configure():
    load_dotenv()

In [127]:
def configure():
    load_dotenv()


Get API data

In [128]:
VALID_ACTION = {'ethsupply2', 'ethprice'}

def get_ethstats(action):
    # Error handling for specific parameter inputs
    if action not in VALID_ACTION:
        raise ValueError("Action must be one of %r." % VALID_ACTION)
    # Load env config
    configure()
    # Define URL and get data
    url = f"https://api.etherscan.io/api?module=stats&action={action}&apikey={os.getenv('api_key')}"
    res = requests.get(url)
    return res.json()

json_SupplyData = get_ethstats('ethsupply2')
json_PriceData = get_ethstats('ethprice')

print(json_PriceData)


{'status': '1', 'message': 'OK', 'result': {'ethbtc': '0.06691', 'ethbtc_timestamp': '1683719253', 'ethusd': '1850.81', 'ethusd_timestamp': '1683719248'}}


In [129]:
keys = ['status','message']

PriceCallStatus = list(map(json_SupplyData.get, keys))
#print(PriceCallStatus)

SupplyCallStatus = list(map(json_PriceData.get, keys))
#print(SupplyCallStatus)
#values = list(map())
    
# if PriceCallStatus[0] != '1':
#     raise ValueError("Status error on ethprice call")
# elif PriceCallStatus[1] != '1':
#     raise ValueError("Auth error on ethprice call")
# elif SupplyCallStatus[0] != 1:
#     raise ValueError("Status error on ethsupply2 call")
# elif SupplyCallStatus[1] != 1:
#     raise ValueError("Auth error on ethsupply2 call")
# else:
#     print('API call success')

Convert API data to Dataframes

In [135]:
df_SupplyData = pd.json_normalize(json_SupplyData['result'])
df_PriceData = pd.json_normalize(json_PriceData['result'])


                     EthSupply                Eth2Staking  \
0  122373866217800000000000000  1187683717196251000000000   

                   BurntFees             WithdrawnTotal  
0  3243849089305856770977483  2484951743300237000000000  
    ethbtc ethbtc_timestamp   ethusd ethusd_timestamp
0  0.06691       1683719253  1850.81       1683719248


In [136]:
CallStatusData = {'PriceCallStatus':PriceCallStatus[0],
                        'PriceCallMessage':PriceCallStatus[1],
                        'SupplyCallStatus':SupplyCallStatus[0],
                        'SupplyCallMessage':SupplyCallStatus[1]}


df_CallStatusData = pd.DataFrame([CallStatusData])